In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

In [26]:
import random
p = 0.1  # 10% of the lines
# keep the header, then take only 1% of lines
# if random from [0,1] interval is greater than 0.01 the row will be skipped
df = pd.read_csv(
         'learning_traces.13m.csv',
         header=0, 
         skiprows=lambda i: i>0 and random.random() > p
)
df['timestamp'] = pd.to_datetime(df['timestamp'],unit='s')
df.drop_duplicates(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1285681 entries, 0 to 1285681
Data columns (total 12 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   p_recall           1285681 non-null  float64       
 1   timestamp          1285681 non-null  datetime64[ns]
 2   delta              1285681 non-null  int64         
 3   user_id            1285681 non-null  object        
 4   learning_language  1285681 non-null  object        
 5   ui_language        1285681 non-null  object        
 6   lexeme_id          1285681 non-null  object        
 7   lexeme_string      1285681 non-null  object        
 8   history_seen       1285681 non-null  int64         
 9   history_correct    1285681 non-null  int64         
 10  session_seen       1285681 non-null  int64         
 11  session_correct    1285681 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(5), object(5)
memory usage: 127.5+ MB


In [24]:
# get parts of speech

def lexeme_df(filename):

    import re
    df_single_col = pd.read_csv(filename, delimiter='\t', header=None, names=['line'])

    def split_line(line):
        parts = re.split(r'\s+', line, maxsplit=2)
        if len(parts) == 3:
            return parts
        return [None, None, None]

    df_split = df_single_col['line'].apply(split_line)
    df = pd.DataFrame(df_split.tolist(), columns=['lexeme', 'category', 'meaning'])

    return df

In [25]:
lexemes = lexeme_df('lexeme_reference.txt')
lexemes.head()

,lexeme,category,meaning
0,aa,animacy,Animate
1,acr,adjective,Acronym
2,adj,POS,Adjective
3,adv,POS,Adverb
4,al,other,Other (altre)


In [28]:
lexemes['lexeme'].nunique()

92

In [27]:
# one-hot encoding
# dummy variables / indicator variables

df.loc[0,"lexeme_string"]

"<det><def><nt><sg><nom>"

# det, df, nt, sg, nom + 87 more 


'das/das<det><def><nt><sg><nom>'